In [1]:
import numpy as np 
import cv2
import tensorflow as tf

2022-05-11 09:15:40.881468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kabeer/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-05-11 09:15:40.881502: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from scipy.special import softmax

In [3]:
# Load all the models 



In [4]:
# face detection model 
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                                './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [5]:
# face mask recognition model (tensorflow model)
model = tf.keras.models.load_model('face_cnn_model/')

2022-05-11 09:15:44.929933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 09:15:44.930254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kabeer/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-05-11 09:15:44.930302: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kabeer/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-05-11 09:15:44.930343: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: lib

In [6]:
# Label Information 

labels = ['Mask' , 'No Mask', 'Covered Mouth and Chin' ,' Covered Nose and Mouth']

def getColor(label):
    if label == "Mask":
        color = (0,255,0)
    elif label == "No Mask":
        color = (0,0,255)
    elif label == "Covered Mouth and Chin":
        color = (0,255,255)
    else:
        color = (255,255,0)
        
    return color

In [7]:
# test color 
getColor('Covered Mouth and Chin')

(0, 255, 255)

# Recognition Code 

In [8]:
# step 1 --> face detection 

# step 2 --> Data Preprocessing 


# step 3 --> Deep Learning(CNN)


In [9]:
# img = cv2.imread(r'C:\Users\asus\Desktop\Face Mask Detection\2.deep_learning\data\Mask_Nose_Mouth\00009.jpg')

# cv2.imshow('original image ',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
def face_mask_prediction(img):
    #step 1 --> face detection 
    image = img.copy()
    h,w = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image , 1 ,(300,300) , (104,117,123) , swapRB = True)

    face_detection_model.setInput(blob)
    detection = face_detection_model.forward() # it will give the detection in 3 to 7 

    for i in range(0,detection.shape[2]):
        confidence = detection[0,0,i,2]
        if confidence > 0.5:
            box = detection[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype(int)
            pt1 = (box[0],box[1])
            pt2 = (box[2],box[3])
    #         cv2.rectangle(image,pt1,pt2,(0,255,0),1)

            # step 2 --> data preprocess

            face = image[box[1]:box[3],box[0]:box[2]]
            face_blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True)
            face_blob_squeeze = np.squeeze(face_blob).T

            face_blob_rotate = cv2.rotate(face_blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
            face_blob_flip = cv2.flip(face_blob_rotate,1)
            #normalization
            img_norm = np.maximum(face_blob_flip,0)/face_blob_flip.max()

            # step 3--> Deep Learning (CNN)
            img_input = img_norm.reshape(1,100,100,3)
            result = model.predict(img_input)
            result = softmax(result)[0]
            confidence_index = result.argmax()
    #         print(confidence_index)
            confidence_score = result[confidence_index]
            label = labels[confidence_index]
            label_text = '{}:{:,.0f}%'.format(label,confidence_score*100)
    #         print(label_text)
            color = getColor(label)
            cv2.rectangle(image,pt1,pt2,color,1)
            cv2.putText(image,label_text,pt1,cv2.FONT_HERSHEY_PLAIN,1,color,1)
            
    return image
        


        

In [11]:
# softmax(result)



In [12]:
# img = cv2.imread(r'C:\Users\asus\Desktop\Face Mask Detection\2.deep_learning\data\Mask\00001.jpg')

# image = face_mask_prediction(img)

# cv2.imshow('original image ',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Real time Face Mask Detection 


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret,frames = cap.read()
    if ret == False:
        break
        
    
    image = face_mask_prediction(frames)
#     cv2.namedWindow("Resized_Window", cv2.WINDOW_NORMAL)
#     cv2.resizeWindow("Resized_Window", 900, 1080)
    cv2.imshow("Resized_Window",image)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()